# OpenPyXL 库学习指南

## 目录
1. [OpenPyXL简介](#1-openpyxl简介)
2. [基础操作：创建工作簿和工作表](#2-基础操作创建工作簿和工作表)
3. [写入数据：从简单到复杂](#3-写入数据从简单到复杂)
4. [读取数据：灵活的数据获取](#4-读取数据灵活的数据获取)
5. [样式设置：美化你的表格](#5-样式设置美化你的表格)
6. [公式和函数：让Excel自动计算](#6-公式和函数让excel自动计算)
7. [批量操作：高效处理大量数据](#7-批量操作高效处理大量数据)
8. [高级功能：超越基础操作](#8-高级功能超越基础操作)
9. [效率提升技巧：自动化你的工作](#9-效率提升技巧自动化你的工作)
10. [实践练习：综合应用](#10-实践练习综合应用)


## 1. OpenPyXL简介

**OpenPyXL** 是Python中用于读写Excel 2010 xlsx/xlsm/xltx/xltm文件的强大库。相比pandas等库，openpyxl提供了更精细的Excel文件控制能力。

### 主要特点：
- **精确控制**：可以控制单元格样式、格式、公式等细节
- **完整功能**：支持图表、数据验证、条件格式等Excel高级功能
- **批量处理**：可以高效处理大量Excel文件
- **自动化**：可以替代手动操作Excel，提高工作效率
- **兼容性好**：生成的Excel文件完全兼容Microsoft Excel

### 安装方法：
```bash
pip install openpyxl
```

### 为什么选择OpenPyXL？
- **vs Pandas**：pandas更适合数据分析，openpyxl更适合精确控制Excel格式和样式
- **vs 直接使用Excel**：可以批量处理、自动化重复工作、程序化生成复杂报表


In [ ]:
# 导入openpyxl库
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter, column_index_from_string
from openpyxl.styles import Font, PatternFill, Border, Side, Alignment
from openpyxl.chart import BarChart, LineChart, PieChart, Reference
from datetime import datetime
import os

# 检查openpyxl版本
import openpyxl
print(f"OpenPyXL版本: {openpyxl.__version__}")


## 2. 基础操作：创建工作簿和工作表

### 2.1 创建新的工作簿

就像在Excel中点击"新建"一样，我们可以用代码创建一个全新的工作簿。工作簿（Workbook）是Excel文件的基础容器，里面可以包含多个工作表（Worksheet）。


In [ ]:
# 创建一个新的工作簿（就像打开Excel并创建新文件）
wb = Workbook()

# 获取默认的活动工作表（Excel默认会有一个Sheet1）
ws = wb.active
print(f"默认工作表名称: {ws.title}")

# 保存工作簿（就像Ctrl+S保存文件）
wb.save("示例1_基础工作簿.xlsx")
print("工作簿已创建并保存！")


### 2.2 创建和操作多个工作表

在实际工作中，我们经常需要在一个Excel文件中创建多个工作表，就像Excel底部的Sheet1、Sheet2标签一样。


In [ ]:
# 创建新工作簿
wb = Workbook()

# 删除默认工作表（如果需要）
# wb.remove(wb.active)

# 创建多个工作表（就像在Excel中点击"+"号添加新工作表）
ws1 = wb.create_sheet("销售数据")
ws2 = wb.create_sheet("库存数据")
ws3 = wb.create_sheet("财务报表", 0)  # 0表示插入到最前面

# 查看所有工作表名称
print("所有工作表:")
for sheet in wb.sheetnames:
    print(f"  - {sheet}")

# 通过名称访问工作表
ws_sales = wb["销售数据"]
print(f"\n当前活动工作表: {wb.active.title}")

wb.save("示例2_多工作表.xlsx")


## 3. 写入数据：从简单到复杂

### 3.1 基础写入：单个单元格

就像在Excel中点击单元格并输入数据一样，我们可以通过代码精确控制每个单元格的内容。


In [ ]:
# 创建新工作簿
wb = Workbook()
ws = wb.active
ws.title = "基础数据写入"

# 方法1：直接通过单元格坐标赋值（最直观）
ws['A1'] = "姓名"
ws['B1'] = "年龄"
ws['C1'] = "城市"

# 方法2：通过行列索引（行和列都从1开始）
ws.cell(row=2, column=1, value="张三")
ws.cell(row=2, column=2, value=25)
ws.cell(row=2, column=3, value="北京")

ws.cell(row=3, column=1, value="李四")
ws.cell(row=3, column=2, value=30)
ws.cell(row=3, column=3, value="上海")

# 方法3：先获取单元格对象，再赋值
cell = ws['A4']
cell.value = "王五"
ws['B4'] = 28
ws['C4'] = "广州"

# 查看数据
print("写入的数据:")
for row in ws.iter_rows(min_row=1, max_row=4, values_only=True):
    print(row)

wb.save("示例3_基础写入.xlsx")


### 3.2 批量写入：使用列表和循环

当需要写入大量数据时，手动一个个单元格输入效率太低。我们可以用循环批量写入，就像Excel的填充功能一样高效。


In [ ]:
# 创建新工作簿
wb = Workbook()
ws = wb.active
ws.title = "批量写入"

# 准备数据（通常这些数据来自数据库、API或其他数据源）
headers = ["产品名称", "单价", "数量", "总价"]
products = [
    ["笔记本电脑", 5999, 5, 29995],
    ["鼠标", 99, 20, 1980],
    ["键盘", 299, 15, 4485],
    ["显示器", 1999, 8, 15992],
    ["音响", 499, 10, 4990]
]

# 批量写入表头
for col_idx, header in enumerate(headers, start=1):
    ws.cell(row=1, column=col_idx, value=header)

# 批量写入数据行
for row_idx, product in enumerate(products, start=2):
    for col_idx, value in enumerate(product, start=1):
        ws.cell(row=row_idx, column=col_idx, value=value)

# 更优雅的方式：使用append方法（一行一行添加）
ws2 = wb.create_sheet("优雅方式")
ws2.append(headers)  # 添加表头
for product in products:
    ws2.append(product)  # 添加数据行

wb.save("示例4_批量写入.xlsx")
print("批量写入完成！")


### 3.3 写入不同类型的数据

Excel可以存储各种类型的数据：文本、数字、日期、时间等。openpyxl可以自动识别并正确存储这些数据类型。


In [ ]:
# 创建新工作簿
wb = Workbook()
ws = wb.active
ws.title = "数据类型示例"

# 写入不同类型的数据
ws['A1'] = "数据类型"
ws['B1'] = "示例值"

# 文本
ws['A2'] = "文本"
ws['B2'] = "这是文本内容"

# 整数
ws['A3'] = "整数"
ws['B3'] = 100

# 浮点数
ws['A4'] = "浮点数"
ws['B4'] = 99.99

# 布尔值
ws['A5'] = "布尔值"
ws['B5'] = True
ws['B6'] = False

# 日期时间
ws['A7'] = "日期"
ws['B7'] = datetime(2024, 1, 15)

ws['A8'] = "日期时间"
ws['B8'] = datetime(2024, 1, 15, 14, 30, 0)

# 公式（后面会详细讲解）
ws['A9'] = "公式"
ws['B9'] = "=SUM(B3:B4)"

# 查看单元格的数据类型
print("单元格数据类型:")
for row in range(2, 10):
    cell = ws[f'B{row}']
    print(f"  B{row}: {cell.value} (类型: {type(cell.value).__name__})")

wb.save("示例5_数据类型.xlsx")


## 4. 读取数据：灵活的数据获取

### 4.1 读取已存在的Excel文件

就像在Excel中打开文件一样，我们可以用代码打开并读取Excel文件中的数据。


In [ ]:
# 首先创建一个示例文件用于读取
wb_write = Workbook()
ws_write = wb_write.active
ws_write.title = "示例数据"

# 写入一些示例数据
data = [
    ["员工ID", "姓名", "部门", "薪资"],
    [1001, "张三", "销售部", 8000],
    [1002, "李四", "技术部", 12000],
    [1003, "王五", "人事部", 7500],
    [1004, "赵六", "技术部", 15000],
]

for row in data:
    ws_write.append(row)

wb_write.save("示例6_读取数据源.xlsx")

# 现在读取这个文件
wb_read = load_workbook("示例6_读取数据源.xlsx")
ws_read = wb_read.active

print(f"工作表名称: {ws_read.title}")
print(f"最大行数: {ws_read.max_row}")
print(f"最大列数: {ws_read.max_column}")

# 读取单个单元格
print(f"\nA1单元格的值: {ws_read['A1'].value}")

# 读取整行
print(f"\n第1行数据: {[cell.value for cell in ws_read[1]]}")

# 读取整列
print(f"\nA列数据: {[cell.value for cell in ws_read['A']]}")

wb_read.close()


### 4.2 遍历读取：高效获取所有数据

当需要读取整个表格的数据时，遍历是最常用的方法。openpyxl提供了多种遍历方式，就像在Excel中拖动选择区域一样灵活。


In [ ]:
# 打开之前创建的文件
wb = load_workbook("示例6_读取数据源.xlsx")
ws = wb.active

print("=== 方法1: 按行遍历（iter_rows）===")
for row in ws.iter_rows(min_row=1, max_row=ws.max_row, values_only=True):
    print(row)

print("\n=== 方法2: 按列遍历（iter_cols）===")
for col in ws.iter_cols(min_col=1, max_col=ws.max_column, values_only=True):
    print(list(col))

print("\n=== 方法3: 遍历指定范围 ===")
# 只读取数据部分（跳过表头）
for row in ws.iter_rows(min_row=2, max_row=ws.max_row, values_only=True):
    print(f"员工: {row[1]}, 部门: {row[2]}, 薪资: {row[3]}")

print("\n=== 方法4: 转换为列表（适合小数据量）===")
all_data = list(ws.iter_rows(values_only=True))
for row in all_data:
    print(row)

wb.close()


## 5. 样式设置：美化你的表格

这是openpyxl相比pandas等库的一大优势！pandas主要关注数据，而openpyxl可以精确控制Excel的每一个视觉细节，让你的报表专业又美观。

### 5.1 字体样式：让文字更醒目

就像在Excel中选中文字后设置字体、大小、颜色一样，我们可以用代码批量设置。


In [ ]:
# 创建新工作簿
wb = Workbook()
ws = wb.active
ws.title = "字体样式示例"

# 写入数据
ws['A1'] = "普通文字"
ws['B1'] = "加粗文字"
ws['C1'] = "斜体文字"
ws['D1'] = "大号文字"
ws['E1'] = "红色文字"

# 设置字体样式
from openpyxl.styles import Font

# 普通文字（默认）
ws['A1'].font = Font()

# 加粗
ws['B1'].font = Font(bold=True)

# 斜体
ws['C1'].font = Font(italic=True)

# 大号字体
ws['D1'].font = Font(size=16)

# 红色文字
ws['E1'].font = Font(color="FF0000")  # RGB颜色，FF0000是红色

# 组合样式：加粗+大号+蓝色
ws['F1'] = "组合样式"
ws['F1'].font = Font(bold=True, size=14, color="0000FF", name="微软雅黑")

# 表头样式：加粗+居中+背景色
ws['A2'] = "产品"
ws['B2'] = "价格"
for cell in ws[2]:
    cell.font = Font(bold=True, size=12)
    cell.alignment = Alignment(horizontal="center", vertical="center")

wb.save("示例7_字体样式.xlsx")
print("字体样式设置完成！")


### 5.2 单元格填充：背景色让表格更清晰

就像在Excel中使用"填充颜色"功能一样，我们可以为单元格设置背景色，让表格层次分明。


In [ ]:
# 创建新工作簿
wb = Workbook()
ws = wb.active
ws.title = "填充样式示例"

# 准备数据
headers = ["月份", "销售额", "增长率"]
data = [
    ["1月", 100000, "5%"],
    ["2月", 120000, "20%"],
    ["3月", 150000, "25%"],
    ["4月", 180000, "20%"],
]

# 写入数据
ws.append(headers)
for row in data:
    ws.append(row)

# 设置表头背景色（浅蓝色）
header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
header_font = Font(bold=True, color="FFFFFF", size=12)

for cell in ws[1]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = Alignment(horizontal="center", vertical="center")

# 设置数据行交替背景色（斑马纹效果）
light_fill = PatternFill(start_color="D9E1F2", end_color="D9E1F2", fill_type="solid")
dark_fill = PatternFill(start_color="B4C6E7", end_color="B4C6E7", fill_type="solid")

for row_idx in range(2, ws.max_row + 1):
    fill = light_fill if row_idx % 2 == 0 else dark_fill
    for cell in ws[row_idx]:
        cell.fill = fill

# 高亮显示增长率超过20%的行
highlight_fill = PatternFill(start_color="FFC000", end_color="FFC000", fill_type="solid")
for row_idx in range(2, ws.max_row + 1):
    growth = ws.cell(row=row_idx, column=3).value
    if growth and float(growth.replace("%", "")) > 20:
        for cell in ws[row_idx]:
            cell.fill = highlight_fill

wb.save("示例8_填充样式.xlsx")
print("填充样式设置完成！")


### 5.3 边框和对齐：让表格更专业

边框让单元格边界清晰，对齐让数据整齐美观。这些细节决定了报表的专业程度。


In [ ]:
# 创建新工作簿
wb = Workbook()
ws = wb.active
ws.title = "边框对齐示例"

# 准备数据
data = [
    ["项目", "Q1", "Q2", "Q3", "Q4", "总计"],
    ["收入", 1000000, 1200000, 1500000, 1800000, 5500000],
    ["成本", 600000, 700000, 850000, 1000000, 3150000],
    ["利润", 400000, 500000, 650000, 800000, 2350000],
]

for row in data:
    ws.append(row)

# 定义边框样式
thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

thick_border = Border(
    left=Side(style='thick'),
    right=Side(style='thick'),
    top=Side(style='thick'),
    bottom=Side(style='thick')
)

# 为所有单元格添加边框
for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
    for cell in row:
        cell.border = thin_border

# 表头加粗边框
for cell in ws[1]:
    cell.border = thick_border

# 设置对齐方式
# 表头：居中
for cell in ws[1]:
    cell.alignment = Alignment(horizontal="center", vertical="center")

# 第一列（项目）：左对齐
for row in range(2, ws.max_row + 1):
    ws.cell(row=row, column=1).alignment = Alignment(horizontal="left", vertical="center")

# 数字列：右对齐，千分位格式
for row in range(2, ws.max_row + 1):
    for col in range(2, ws.max_column + 1):
        cell = ws.cell(row=row, column=col)
        cell.alignment = Alignment(horizontal="right", vertical="center")
        if isinstance(cell.value, (int, float)):
            cell.number_format = '#,##0'  # 千分位格式

# 调整列宽
ws.column_dimensions['A'].width = 12
for col in range(2, ws.max_column + 1):
    ws.column_dimensions[get_column_letter(col)].width = 15

wb.save("示例9_边框对齐.xlsx")
print("边框和对齐设置完成！")


## 6. 公式和函数：让Excel自动计算

这是openpyxl相比直接操作Excel的一大优势！我们可以批量设置公式，而不需要手动在Excel中一个个输入。

### 6.1 基础公式：SUM、AVERAGE等

就像在Excel中输入"=SUM(A1:A10)"一样，我们可以用代码设置公式。


In [ ]:
# 创建新工作簿
wb = Workbook()
ws = wb.active
ws.title = "公式示例"

# 准备数据
data = [
    ["月份", "销售额", "成本", "利润"],
    ["1月", 100000, 60000, None],  # 利润用公式计算
    ["2月", 120000, 70000, None],
    ["3月", 150000, 85000, None],
    ["4月", 180000, 100000, None],
    ["总计", None, None, None],
]

# 写入数据
for row in data:
    ws.append(row)

# 设置利润公式（利润 = 销售额 - 成本）
for row in range(2, 6):  # 第2-5行是数据行
    ws.cell(row=row, column=4).value = f"=B{row}-C{row}"

# 设置总计公式
ws.cell(row=6, column=2).value = "=SUM(B2:B5)"  # 销售额总计
ws.cell(row=6, column=3).value = "=SUM(C2:C5)"  # 成本总计
ws.cell(row=6, column=4).value = "=SUM(D2:D5)"  # 利润总计

# 设置平均值
ws['A8'] = "平均销售额"
ws['B8'] = "=AVERAGE(B2:B5)"

ws['A9'] = "平均利润"
ws['B9'] = "=AVERAGE(D2:D5)"

# 设置最大值和最小值
ws['A10'] = "最高销售额"
ws['B10'] = "=MAX(B2:B5)"

ws['A11'] = "最低销售额"
ws['B11'] = "=MIN(B2:B5)"

# 格式化数字
for row in range(2, 7):
    for col in range(2, 5):
        cell = ws.cell(row=row, column=col)
        if cell.value and not str(cell.value).startswith('='):
            cell.number_format = '#,##0'

wb.save("示例10_公式计算.xlsx")
print("公式设置完成！注意：公式在Excel中打开时会自动计算。")


### 6.2 条件函数：IF、VLOOKUP等

Excel的条件函数非常强大，我们可以用代码批量设置这些逻辑判断公式。


In [ ]:
# 创建新工作簿
wb = Workbook()
ws = wb.active
ws.title = "条件函数示例"

# 准备销售数据
headers = ["员工", "销售额", "目标", "完成率", "评级", "奖金"]
data = [
    ["张三", 120000, 100000, None, None, None],
    ["李四", 80000, 100000, None, None, None],
    ["王五", 150000, 100000, None, None, None],
    ["赵六", 95000, 100000, None, None, None],
    ["钱七", 110000, 100000, None, None, None],
]

ws.append(headers)
for row in data:
    ws.append(row)

# 设置完成率公式（销售额/目标）
for row in range(2, ws.max_row + 1):
    ws.cell(row=row, column=4).value = f"=B{row}/C{row}"
    ws.cell(row=row, column=4).number_format = "0.00%"  # 百分比格式

# 设置评级公式（IF函数：完成率>=100%为"优秀"，否则为"待改进"）
for row in range(2, ws.max_row + 1):
    ws.cell(row=row, column=5).value = f'=IF(D{row}>=1,"优秀","待改进")'

# 设置奖金公式（嵌套IF：销售额>=120000奖金5000，>=100000奖金3000，否则1000）
for row in range(2, ws.max_row + 1):
    ws.cell(row=row, column=6).value = f'=IF(B{row}>=120000,5000,IF(B{row}>=100000,3000,1000))'

# 格式化奖金列
for row in range(2, ws.max_row + 1):
    ws.cell(row=row, column=6).number_format = '#,##0'

# 调整列宽
for col in range(1, ws.max_column + 1):
    ws.column_dimensions[get_column_letter(col)].width = 12

wb.save("示例11_条件函数.xlsx")
print("条件函数设置完成！")


## 7. 批量操作：高效处理大量数据

这是openpyxl相比手动操作Excel的最大优势！我们可以用几行代码完成需要手动操作几百次的工作。

### 7.1 批量格式化：一键美化整个表格

想象一下，如果你有100行数据需要设置格式，手动操作需要点击几百次。用代码只需要几秒钟！


In [ ]:
# 创建包含大量数据的工作簿
wb = Workbook()
ws = wb.active
ws.title = "批量格式化示例"

# 生成100行数据
import random
headers = ["日期", "产品", "数量", "单价", "总价", "销售员"]
ws.append(headers)

products = ["笔记本", "鼠标", "键盘", "显示器", "音响"]
salesmen = ["张三", "李四", "王五", "赵六"]

for i in range(100):
    date = datetime(2024, 1, 1 + (i % 30), 0, 0, 0)
    product = random.choice(products)
    quantity = random.randint(1, 20)
    price = random.randint(50, 5000)
    total = quantity * price
    salesman = random.choice(salesmen)
    ws.append([date, product, quantity, price, total, salesman])

# 批量设置表头样式
header_fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
header_font = Font(bold=True, color="FFFFFF", size=11)
header_alignment = Alignment(horizontal="center", vertical="center")
thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

for cell in ws[1]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = header_alignment
    cell.border = thin_border

# 批量设置数据行样式
for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
    for cell in row:
        cell.border = thin_border
        
        # 数字列右对齐
        if cell.column in [3, 4, 5]:
            cell.alignment = Alignment(horizontal="right", vertical="center")
            if isinstance(cell.value, (int, float)):
                cell.number_format = '#,##0'
        else:
            cell.alignment = Alignment(horizontal="left", vertical="center")
        
        # 日期格式
        if isinstance(cell.value, datetime):
            cell.number_format = 'YYYY-MM-DD'

# 斑马纹效果
light_fill = PatternFill(start_color="F2F2F2", end_color="F2F2F2", fill_type="solid")
for row_idx in range(2, ws.max_row + 1):
    if row_idx % 2 == 0:
        for cell in ws[row_idx]:
            cell.fill = light_fill

# 自动调整列宽
for col in range(1, ws.max_column + 1):
    max_length = 0
    column_letter = get_column_letter(col)
    for cell in ws[column_letter]:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(str(cell.value))
        except:
            pass
    adjusted_width = min(max_length + 2, 50)
    ws.column_dimensions[column_letter].width = adjusted_width

wb.save("示例12_批量格式化.xlsx")
print(f"批量格式化完成！共处理 {ws.max_row} 行数据。")


### 7.2 批量处理多个文件：自动化报表生成

假设你需要每天处理10个Excel文件，手动操作需要1小时，用代码只需要1分钟！


In [ ]:
# 模拟场景：批量处理多个部门的销售报表
import os

# 创建多个部门的原始数据文件
departments = ["销售部", "市场部", "技术部", "人事部"]
base_data = {
    "销售部": [["产品", "销售额"], ["A产品", 100000], ["B产品", 150000], ["C产品", 120000]],
    "市场部": [["活动", "费用"], ["活动1", 50000], ["活动2", 80000], ["活动3", 60000]],
    "技术部": [["项目", "成本"], ["项目1", 200000], ["项目2", 180000], ["项目3", 220000]],
    "人事部": [["类别", "支出"], ["薪资", 300000], ["培训", 50000], ["福利", 40000]],
}

# 创建原始文件
for dept in departments:
    wb = Workbook()
    ws = wb.active
    ws.title = dept
    for row in base_data[dept]:
        ws.append(row)
    wb.save(f"原始_{dept}.xlsx")

print("原始文件创建完成！")

# 批量处理：为每个文件添加总计、格式化、保存
def process_excel_file(filename):
    """处理单个Excel文件的函数"""
    wb = load_workbook(filename)
    ws = wb.active
    
    # 添加总计行
    last_row = ws.max_row
    last_col = ws.max_column
    
    # 在最后一行添加"总计"
    ws.cell(row=last_row + 1, column=1, value="总计")
    
    # 为数字列添加SUM公式
    for col in range(2, last_col + 1):
        ws.cell(row=last_row + 1, column=col).value = f"=SUM({get_column_letter(col)}2:{get_column_letter(col)}{last_row})"
        ws.cell(row=last_row + 1, column=col).number_format = '#,##0'
    
    # 格式化表头
    header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
    header_font = Font(bold=True, color="FFFFFF")
    for cell in ws[1]:
        cell.fill = header_fill
        cell.font = header_font
        cell.alignment = Alignment(horizontal="center")
    
    # 格式化总计行
    total_fill = PatternFill(start_color="FFC000", end_color="FFC000", fill_type="solid")
    total_font = Font(bold=True)
    for cell in ws[last_row + 1]:
        cell.fill = total_fill
        cell.font = total_font
    
    # 保存处理后的文件
    new_filename = filename.replace("原始_", "处理后的_")
    wb.save(new_filename)
    wb.close()
    return new_filename

# 批量处理所有文件
processed_files = []
for dept in departments:
    filename = f"原始_{dept}.xlsx"
    if os.path.exists(filename):
        new_file = process_excel_file(filename)
        processed_files.append(new_file)
        print(f"✓ 已处理: {filename} -> {new_file}")

print(f"\n批量处理完成！共处理 {len(processed_files)} 个文件。")


### 7.3 数据合并：将多个工作表合并为一个

在实际工作中，我们经常需要将多个部门的数据合并到一个总表中，手动复制粘贴非常耗时。


In [ ]:
# 创建多个部门的数据文件（模拟不同部门的数据）
wb1 = Workbook()
ws1 = wb1.active
ws1.append(["部门", "员工", "销售额"])
ws1.append(["销售部", "张三", 50000])
ws1.append(["销售部", "李四", 60000])
wb1.save("部门1_销售数据.xlsx")
wb1.close()

wb2 = Workbook()
ws2 = wb2.active
ws2.append(["部门", "员工", "销售额"])
ws2.append(["市场部", "王五", 40000])
ws2.append(["市场部", "赵六", 45000])
wb2.save("部门2_销售数据.xlsx")
wb2.close()

wb3 = Workbook()
ws3 = wb3.active
ws3.append(["部门", "员工", "销售额"])
ws3.append(["技术部", "钱七", 30000])
ws3.append(["技术部", "孙八", 35000])
wb3.save("部门3_销售数据.xlsx")
wb3.close()

# 合并多个文件的数据到一个总表
def merge_excel_files(file_list, output_file):
    """合并多个Excel文件的数据"""
    merged_wb = Workbook()
    merged_ws = merged_wb.active
    merged_ws.title = "合并数据"
    
    header_written = False
    
    for file in file_list:
        if os.path.exists(file):
            wb = load_workbook(file)
            ws = wb.active
            
            # 读取数据（跳过表头或包含表头）
            for row_idx, row in enumerate(ws.iter_rows(values_only=True), start=1):
                # 第一次写入表头
                if not header_written and row_idx == 1:
                    merged_ws.append(row)
                    header_written = True
                # 写入数据行（跳过第一个文件的表头）
                elif row_idx > 1 or header_written:
                    merged_ws.append(row)
            
            wb.close()
    
    # 格式化合并后的数据
    header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
    header_font = Font(bold=True, color="FFFFFF")
    
    for cell in merged_ws[1]:
        cell.fill = header_fill
        cell.font = header_font
        cell.alignment = Alignment(horizontal="center")
    
    # 添加边框
    thin_border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )
    
    for row in merged_ws.iter_rows():
        for cell in row:
            cell.border = thin_border
    
    merged_wb.save(output_file)
    merged_wb.close()
    return output_file

# 执行合并
files_to_merge = ["部门1_销售数据.xlsx", "部门2_销售数据.xlsx", "部门3_销售数据.xlsx"]
merged_file = merge_excel_files(files_to_merge, "合并后的总表.xlsx")

print("数据合并完成！")
print(f"合并文件: {merged_file}")

# 查看合并后的数据
wb_check = load_workbook(merged_file)
ws_check = wb_check.active
print(f"\n合并后的数据（共 {ws_check.max_row} 行）:")
for row in ws_check.iter_rows(values_only=True):
    print(row)
wb_check.close()


## 8. 高级功能：超越基础操作

openpyxl提供了许多Excel的高级功能，这些功能用代码实现比手动操作更高效、更准确。

### 8.1 创建图表：可视化数据

在Excel中创建图表需要多个步骤，用代码可以一键生成专业的图表。


In [ ]:
# 创建包含图表的工作簿
wb = Workbook()
ws = wb.active
ws.title = "销售数据"

# 准备数据
data = [
    ["月份", "销售额", "成本", "利润"],
    ["1月", 100000, 60000, 40000],
    ["2月", 120000, 70000, 50000],
    ["3月", 150000, 85000, 65000],
    ["4月", 180000, 100000, 80000],
    ["5月", 200000, 110000, 90000],
    ["6月", 220000, 120000, 100000],
]

for row in data:
    ws.append(row)

# 格式化数据
header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
header_font = Font(bold=True, color="FFFFFF")

for cell in ws[1]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = Alignment(horizontal="center")

# 创建柱状图
chart1 = BarChart()
chart1.type = "col"  # 柱状图
chart1.style = 10
chart1.title = "月度销售额对比"
chart1.y_axis.title = "金额"
chart1.x_axis.title = "月份"

# 设置数据范围
data_ref = Reference(ws, min_col=2, min_row=1, max_row=7, max_col=2)  # 销售额列
cats_ref = Reference(ws, min_col=1, min_row=2, max_row=7)  # 月份列

chart1.add_data(data_ref, titles_from_data=True)
chart1.set_categories(cats_ref)

# 将图表添加到工作表
ws.add_chart(chart1, "E2")

# 创建折线图（利润趋势）
chart2 = LineChart()
chart2.title = "利润趋势"
chart2.y_axis.title = "利润"
chart2.x_axis.title = "月份"
chart2.style = 13

data_ref2 = Reference(ws, min_col=4, min_row=1, max_row=7, max_col=4)
chart2.add_data(data_ref2, titles_from_data=True)
chart2.set_categories(cats_ref)

ws.add_chart(chart2, "E18")

# 创建饼图（6月各项占比）
ws2 = wb.create_sheet("饼图示例")
ws2.append(["项目", "金额"])
ws2.append(["销售额", 220000])
ws2.append(["成本", 120000])
ws2.append(["利润", 100000])

chart3 = PieChart()
chart3.title = "6月财务构成"

data_ref3 = Reference(ws2, min_col=2, min_row=1, max_row=4)
cats_ref3 = Reference(ws2, min_col=1, min_row=2, max_row=4)

chart3.add_data(data_ref3, titles_from_data=True)
chart3.set_categories(cats_ref3)

ws2.add_chart(chart3, "D2")

wb.save("示例13_图表.xlsx")
print("图表创建完成！")


### 8.2 条件格式：自动高亮重要数据

条件格式可以让Excel根据数据值自动改变单元格样式，比如自动标红负数、高亮最大值等。用代码可以批量设置复杂的条件格式规则。


In [ ]:
# 注意：openpyxl对条件格式的支持有限，这里展示基础用法
# 更复杂的条件格式建议在Excel中设置，或使用其他方法

wb = Workbook()
ws = wb.active
ws.title = "条件格式示例"

# 准备数据
data = [
    ["产品", "销售额", "目标", "完成率"],
    ["产品A", 120000, 100000, None],
    ["产品B", 80000, 100000, None],
    ["产品C", 150000, 100000, None],
    ["产品D", 95000, 100000, None],
    ["产品E", 110000, 100000, None],
]

for row in data:
    ws.append(row)

# 计算完成率
for row in range(2, ws.max_row + 1):
    ws.cell(row=row, column=4).value = f"=B{row}/C{row}"
    ws.cell(row=row, column=4).number_format = "0.00%"

# 使用Python逻辑实现条件格式效果（在写入时判断）
green_fill = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")
red_fill = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")
yellow_fill = PatternFill(start_color="FFEB9C", end_color="FFEB9C", fill_type="solid")

for row in range(2, ws.max_row + 1):
    sales = ws.cell(row=row, column=2).value
    target = ws.cell(row=row, column=3).value
    
    if sales and target:
        completion_rate = sales / target
        
        # 根据完成率设置背景色
        if completion_rate >= 1.0:
            fill = green_fill  # 完成目标：绿色
        elif completion_rate >= 0.8:
            fill = yellow_fill  # 接近目标：黄色
        else:
            fill = red_fill  # 未达标：红色
        
        # 为整行设置背景色
        for col in range(1, ws.max_column + 1):
            ws.cell(row=row, column=col).fill = fill

# 格式化表头
header_fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
header_font = Font(bold=True, color="FFFFFF")

for cell in ws[1]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = Alignment(horizontal="center")

wb.save("示例14_条件格式.xlsx")
print("条件格式设置完成！")
print("说明：openpyxl对Excel原生条件格式支持有限，这里用Python逻辑实现了类似效果。")


### 8.3 数据验证：限制输入范围

数据验证可以确保用户只能输入有效的数据，比如只能输入1-100之间的数字，或者只能从下拉列表中选择。用代码可以批量设置验证规则。


In [ ]:
from openpyxl.worksheet.datavalidation import DataValidation

wb = Workbook()
ws = wb.active
ws.title = "数据验证示例"

# 准备数据
ws.append(["员工姓名", "部门", "年龄", "评分"])
ws.append(["张三", None, None, None])
ws.append(["李四", None, None, None])
ws.append(["王五", None, None, None])

# 数据验证1：部门下拉列表
dept_validation = DataValidation(type="list", formula1='"销售部,市场部,技术部,人事部"')
dept_validation.error = "请从下拉列表中选择部门"
dept_validation.errorTitle = "输入错误"
dept_validation.prompt = "请选择部门"
dept_validation.promptTitle = "部门选择"

# 应用到B列（部门列）
dept_validation.add("B2:B10")
ws.add_data_validation(dept_validation)

# 数据验证2：年龄范围（18-65）
age_validation = DataValidation(type="whole", operator="between", formula1=18, formula2=65)
age_validation.error = "年龄必须在18-65之间"
age_validation.errorTitle = "输入错误"
age_validation.prompt = "请输入18-65之间的整数"
age_validation.promptTitle = "年龄输入"

# 应用到C列（年龄列）
age_validation.add("C2:C10")
ws.add_data_validation(age_validation)

# 数据验证3：评分范围（1-10，允许小数）
score_validation = DataValidation(type="decimal", operator="between", formula1=1, formula2=10)
score_validation.error = "评分必须在1-10之间"
score_validation.errorTitle = "输入错误"
score_validation.prompt = "请输入1-10之间的数字"
score_validation.promptTitle = "评分输入"

# 应用到D列（评分列）
score_validation.add("D2:D10")
ws.add_data_validation(score_validation)

# 格式化表头
header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
header_font = Font(bold=True, color="FFFFFF")

for cell in ws[1]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = Alignment(horizontal="center")

# 添加说明
ws['F1'] = "数据验证说明："
ws['F2'] = "1. 部门列：只能从下拉列表选择"
ws['F3'] = "2. 年龄列：只能输入18-65的整数"
ws['F4'] = "3. 评分列：只能输入1-10的数字"

wb.save("示例15_数据验证.xlsx")
print("数据验证设置完成！")
print("提示：在Excel中打开文件，尝试在验证列输入无效数据，会看到错误提示。")


### 8.4 冻结窗格：固定表头

当表格很长时，滚动查看数据时表头会消失。冻结窗格可以让表头始终可见，就像Excel中的"冻结首行"功能。


In [ ]:
# 创建包含大量数据的工作簿
wb = Workbook()
ws = wb.active
ws.title = "冻结窗格示例"

# 生成50行数据
ws.append(["序号", "产品名称", "类别", "单价", "库存", "总价值"])

import random
categories = ["电子产品", "办公用品", "生活用品"]
products = ["笔记本", "鼠标", "键盘", "显示器", "音响", "打印机", "扫描仪"]

for i in range(1, 51):
    product = random.choice(products)
    category = random.choice(categories)
    price = random.randint(50, 5000)
    stock = random.randint(0, 100)
    total_value = price * stock
    ws.append([i, product, category, price, stock, total_value])

# 冻结首行（表头）
ws.freeze_panes = "A2"  # 冻结A2以上的行（即第1行）

# 也可以冻结首行和首列
# ws.freeze_panes = "B2"  # 冻结第1行和第1列

# 格式化
header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
header_font = Font(bold=True, color="FFFFFF")

for cell in ws[1]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = Alignment(horizontal="center")

# 调整列宽
for col in range(1, ws.max_column + 1):
    ws.column_dimensions[get_column_letter(col)].width = 15

wb.save("示例16_冻结窗格.xlsx")
print("冻结窗格设置完成！")
print("提示：在Excel中打开文件，向下滚动时表头会保持可见。")


## 9. 效率提升技巧：自动化你的工作

这些技巧可以让你用几行代码完成需要手动操作几小时的工作，真正体现openpyxl的价值！

### 9.1 批量生成报表：一键生成月度报表

假设你需要每月生成10个部门的报表，每个报表需要设置格式、添加公式、创建图表，手动操作需要2小时，用代码只需要2分钟！


In [ ]:
def create_monthly_report(dept_name, month_data, output_file):
    """
    创建月度报表的通用函数
    dept_name: 部门名称
    month_data: 月度数据列表，格式：[["项目", "金额"], ["项目1", 1000], ...]
    output_file: 输出文件名
    """
    wb = Workbook()
    ws = wb.active
    ws.title = f"{dept_name}报表"
    
    # 写入数据
    for row in month_data:
        ws.append(row)
    
    # 添加总计行
    last_row = ws.max_row
    last_col = ws.max_column
    
    ws.cell(row=last_row + 1, column=1, value="总计")
    for col in range(2, last_col + 1):
        ws.cell(row=last_row + 1, column=col).value = f"=SUM({get_column_letter(col)}2:{get_column_letter(col)}{last_row})"
        ws.cell(row=last_row + 1, column=col).number_format = '#,##0'
    
    # 统一格式化
    header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
    header_font = Font(bold=True, color="FFFFFF", size=12)
    header_alignment = Alignment(horizontal="center", vertical="center")
    
    for cell in ws[1]:
        cell.fill = header_fill
        cell.font = header_font
        cell.alignment = header_alignment
    
    total_fill = PatternFill(start_color="FFC000", end_color="FFC000", fill_type="solid")
    total_font = Font(bold=True, size=11)
    
    for cell in ws[last_row + 1]:
        cell.fill = total_fill
        cell.font = total_font
    
    # 添加边框
    thin_border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )
    
    for row in ws.iter_rows():
        for cell in row:
            cell.border = thin_border
            if cell.column > 1 and isinstance(cell.value, (int, float)):
                cell.number_format = '#,##0'
                cell.alignment = Alignment(horizontal="right")
    
    # 自动调整列宽
    for col in range(1, ws.max_column + 1):
        max_length = 0
        column_letter = get_column_letter(col)
        for cell in ws[column_letter]:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        ws.column_dimensions[column_letter].width = min(max_length + 2, 30)
    
    wb.save(output_file)
    wb.close()
    return output_file

# 批量生成多个部门的报表
departments_data = {
    "销售部": [["项目", "金额"], ["产品A", 100000], ["产品B", 150000], ["产品C", 120000]],
    "市场部": [["活动", "费用"], ["线上推广", 50000], ["线下活动", 80000], ["广告投放", 60000]],
    "技术部": [["项目", "成本"], ["开发项目1", 200000], ["开发项目2", 180000], ["维护", 50000]],
    "人事部": [["类别", "支出"], ["员工薪资", 300000], ["培训费用", 50000], ["福利支出", 40000]],
}

generated_files = []
for dept, data in departments_data.items():
    filename = f"2024年1月_{dept}_报表.xlsx"
    create_monthly_report(dept, data, filename)
    generated_files.append(filename)
    print(f"✓ 已生成: {filename}")

print(f"\n批量报表生成完成！共生成 {len(generated_files)} 个报表文件。")
print("如果手动操作，每个报表需要10-15分钟，4个报表需要40-60分钟。")
print("用代码只需要几秒钟！这就是自动化的威力！")


### 9.2 数据清洗和转换：自动化数据处理

从数据库或API获取的数据往往需要清洗和转换才能用于Excel报表。用代码可以自动完成这些重复性工作。


In [ ]:
# 模拟从数据库或API获取的原始数据（通常格式不规范）
raw_data = [
    {"name": " 张三 ", "age": "25", "salary": "8000.50", "dept": "销售部", "date": "2024-01-15"},
    {"name": "李四", "age": "30", "salary": "12000", "dept": "技术部", "date": "2024/01/20"},
    {"name": " 王五 ", "age": "28", "salary": "9500.00", "dept": "市场部", "date": "2024-01-25"},
    {"name": "赵六", "age": "35", "salary": "15000.5", "dept": "技术部", "date": "2024-02-01"},
    {"name": "钱七", "age": "26", "salary": "7500", "dept": "人事部", "date": "2024/02/10"},
]

def clean_and_convert_data(raw_data):
    """清洗和转换数据"""
    cleaned_data = []
    
    for item in raw_data:
        cleaned_item = {}
        
        # 清洗姓名：去除首尾空格
        cleaned_item["姓名"] = item["name"].strip()
        
        # 转换年龄：字符串转整数
        cleaned_item["年龄"] = int(item["age"])
        
        # 转换薪资：字符串转浮点数，保留2位小数
        cleaned_item["薪资"] = round(float(item["salary"]), 2)
        
        # 部门保持不变
        cleaned_item["部门"] = item["dept"]
        
        # 统一日期格式：处理不同的日期格式
        date_str = item["date"].replace("/", "-")
        cleaned_item["入职日期"] = date_str
        
        cleaned_data.append(cleaned_item)
    
    return cleaned_data

# 清洗数据
cleaned_data = clean_and_convert_data(raw_data)

# 创建Excel文件
wb = Workbook()
ws = wb.active
ws.title = "清洗后的数据"

# 写入表头
headers = ["姓名", "年龄", "薪资", "部门", "入职日期"]
ws.append(headers)

# 写入清洗后的数据
for item in cleaned_data:
    ws.append([item["姓名"], item["年龄"], item["薪资"], item["部门"], item["入职日期"]])

# 添加统计信息
ws.append([])  # 空行
ws.append(["统计", "数值"])
ws.append(["总人数", f"=COUNT(B2:B{ws.max_row-2})"])
ws.append(["平均年龄", f"=AVERAGE(B2:B{ws.max_row-3})"])
ws.append(["平均薪资", f"=AVERAGE(C2:C{ws.max_row-4})"])
ws.append(["最高薪资", f"=MAX(C2:C{ws.max_row-5})"])
ws.append(["最低薪资", f"=MIN(C2:C{ws.max_row-6})"])

# 格式化
header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
header_font = Font(bold=True, color="FFFFFF")

for cell in ws[1]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = Alignment(horizontal="center")

# 格式化数字
for row in range(2, ws.max_row - 6):
    ws.cell(row=row, column=3).number_format = '#,##0.00'  # 薪资格式

# 格式化统计部分
stat_fill = PatternFill(start_color="FFC000", end_color="FFC000", fill_type="solid")
stat_font = Font(bold=True)

for row in range(ws.max_row - 6, ws.max_row + 1):
    ws.cell(row=row, column=1).fill = stat_fill
    ws.cell(row=row, column=1).font = stat_font

wb.save("示例17_数据清洗.xlsx")
print("数据清洗和转换完成！")
print("原始数据中的问题已自动修复：")
print("  - 姓名首尾空格已去除")
print("  - 年龄已转换为整数")
print("  - 薪资已统一格式并保留2位小数")
print("  - 日期格式已统一")


### 9.3 模板填充：基于模板批量生成文件

在实际工作中，我们经常有标准的报表模板，只需要填充不同的数据。用代码可以基于一个模板快速生成多个文件。


In [ ]:
# 首先创建一个模板文件
template_wb = Workbook()
template_ws = template_wb.active
template_ws.title = "员工信息表"

# 创建模板结构
template_ws['A1'] = "员工信息表"
template_ws['A1'].font = Font(bold=True, size=16)
template_ws.merge_cells('A1:D1')  # 合并单元格
template_ws['A1'].alignment = Alignment(horizontal="center")

template_ws['A3'] = "姓名："
template_ws['B3'] = ""  # 待填充
template_ws['A4'] = "部门："
template_ws['B4'] = ""  # 待填充
template_ws['A5'] = "职位："
template_ws['B5'] = ""  # 待填充
template_ws['A6'] = "入职日期："
template_ws['B6'] = ""  # 待填充

template_ws['A8'] = "月度绩效"
template_ws['A8'].font = Font(bold=True, size=12)

template_ws['A9'] = "月份"
template_ws['B9'] = "销售额"
template_ws['C9'] = "目标"
template_ws['D9'] = "完成率"

# 格式化模板
header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
header_font = Font(bold=True, color="FFFFFF")

for cell in template_ws[9]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = Alignment(horizontal="center")

template_wb.save("员工信息模板.xlsx")
template_wb.close()

print("模板文件已创建：员工信息模板.xlsx")

# 准备要填充的数据
employees_data = [
    {
        "name": "张三",
        "dept": "销售部",
        "position": "销售经理",
        "join_date": "2020-01-15",
        "performance": [
            ["1月", 100000, 80000],
            ["2月", 120000, 100000],
            ["3月", 150000, 120000],
        ]
    },
    {
        "name": "李四",
        "dept": "市场部",
        "position": "市场专员",
        "join_date": "2021-03-20",
        "performance": [
            ["1月", 80000, 70000],
            ["2月", 90000, 80000],
            ["3月", 110000, 90000],
        ]
    },
    {
        "name": "王五",
        "dept": "技术部",
        "position": "高级工程师",
        "join_date": "2019-06-10",
        "performance": [
            ["1月", 0, 0],  # 技术部可能不考核销售额
            ["2月", 0, 0],
            ["3月", 0, 0],
        ]
    },
]

# 基于模板生成多个文件
def fill_template(template_file, employee_data, output_file):
    """基于模板填充数据"""
    wb = load_workbook(template_file)
    ws = wb.active
    
    # 填充基本信息
    ws['B3'] = employee_data["name"]
    ws['B4'] = employee_data["dept"]
    ws['B5'] = employee_data["position"]
    ws['B6'] = employee_data["join_date"]
    
    # 填充绩效数据
    start_row = 10
    for perf in employee_data["performance"]:
        ws.cell(row=start_row, column=1).value = perf[0]
        ws.cell(row=start_row, column=2).value = perf[1]
        ws.cell(row=start_row, column=3).value = perf[2]
        
        # 计算完成率
        if perf[2] > 0:
            ws.cell(row=start_row, column=4).value = f"=B{start_row}/C{start_row}"
            ws.cell(row=start_row, column=4).number_format = "0.00%"
        else:
            ws.cell(row=start_row, column=4).value = "N/A"
        
        start_row += 1
    
    # 格式化数据行
    thin_border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )
    
    for row in ws.iter_rows(min_row=9, max_row=start_row-1):
        for cell in row:
            cell.border = thin_border
            if cell.column in [2, 3, 4] and isinstance(cell.value, (int, float)):
                cell.alignment = Alignment(horizontal="right")
                if not str(cell.value).startswith('='):
                    cell.number_format = '#,##0'
    
    wb.save(output_file)
    wb.close()
    return output_file

# 批量生成
generated_files = []
for emp in employees_data:
    filename = f"员工信息_{emp['name']}.xlsx"
    fill_template("员工信息模板.xlsx", emp, filename)
    generated_files.append(filename)
    print(f"✓ 已生成: {filename}")

print(f"\n基于模板批量生成完成！共生成 {len(generated_files)} 个文件。")


## 10. 实践练习：综合应用

现在让我们通过一个完整的实际案例，综合运用前面学到的所有知识。

### 练习：创建完整的销售报表系统

任务：创建一个包含多个工作表的销售报表，包括：
1. 原始数据表
2. 汇总统计表
3. 图表分析
4. 格式化美化


In [ ]:
# 创建完整的销售报表系统
wb = Workbook()

# 删除默认工作表
wb.remove(wb.active)

# ========== 工作表1：原始销售数据 ==========
ws_data = wb.create_sheet("销售数据", 0)

# 准备数据
import random
from datetime import datetime, timedelta

headers = ["日期", "销售员", "产品", "数量", "单价", "总价", "地区"]
ws_data.append(headers)

# 生成模拟数据
salesmen = ["张三", "李四", "王五", "赵六", "钱七"]
products = ["笔记本", "鼠标", "键盘", "显示器", "音响"]
regions = ["北京", "上海", "广州", "深圳"]

start_date = datetime(2024, 1, 1)
for i in range(100):  # 生成100条销售记录
    date = start_date + timedelta(days=random.randint(0, 90))
    salesman = random.choice(salesmen)
    product = random.choice(products)
    quantity = random.randint(1, 20)
    price = random.randint(50, 5000)
    total = quantity * price
    region = random.choice(regions)
    ws_data.append([date, salesman, product, quantity, price, total, region])

# 格式化数据表
header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
header_font = Font(bold=True, color="FFFFFF", size=11)
header_alignment = Alignment(horizontal="center", vertical="center")

for cell in ws_data[1]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = header_alignment

thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

for row in ws_data.iter_rows(min_row=2):
    for cell in row:
        cell.border = thin_border
        if cell.column in [4, 5, 6]:  # 数量、单价、总价
            cell.alignment = Alignment(horizontal="right")
            if isinstance(cell.value, (int, float)):
                cell.number_format = '#,##0'
        elif cell.column == 1:  # 日期
            if isinstance(cell.value, datetime):
                cell.number_format = 'YYYY-MM-DD'

# 冻结窗格
ws_data.freeze_panes = "A2"

# ========== 工作表2：汇总统计 ==========
ws_summary = wb.create_sheet("汇总统计")

# 按销售员汇总
ws_summary['A1'] = "按销售员汇总"
ws_summary['A1'].font = Font(bold=True, size=14)
ws_summary['A3'] = "销售员"
ws_summary['B3'] = "销售笔数"
ws_summary['C3'] = "总销售额"
ws_summary['D3'] = "平均单价"

row = 4
for salesman in salesmen:
    ws_summary.cell(row=row, column=1).value = salesman
    ws_summary.cell(row=row, column=2).value = f'=COUNTIF(销售数据.B:B,"{salesman}")'
    ws_summary.cell(row=row, column=3).value = f'=SUMIF(销售数据.B:B,"{salesman}",销售数据.F:F)'
    ws_summary.cell(row=row, column=4).value = f'=C{row}/B{row}'
    row += 1

# 按产品汇总
ws_summary['A10'] = "按产品汇总"
ws_summary['A10'].font = Font(bold=True, size=14)
ws_summary['A12'] = "产品"
ws_summary['B12'] = "销售数量"
ws_summary['C12'] = "总销售额"

row = 13
for product in products:
    ws_summary.cell(row=row, column=1).value = product
    ws_summary.cell(row=row, column=2).value = f'=SUMIF(销售数据.C:C,"{product}",销售数据.D:D)'
    ws_summary.cell(row=row, column=3).value = f'=SUMIF(销售数据.C:C,"{product}",销售数据.F:F)'
    row += 1

# 按地区汇总
ws_summary['A20'] = "按地区汇总"
ws_summary['A20'].font = Font(bold=True, size=14)
ws_summary['A22'] = "地区"
ws_summary['B22'] = "总销售额"

row = 23
for region in regions:
    ws_summary.cell(row=row, column=1).value = region
    ws_summary.cell(row=row, column=2).value = f'=SUMIF(销售数据.G:G,"{region}",销售数据.F:F)'
    row += 1

# 格式化汇总表
for cell in ws_summary[3]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = header_alignment

for cell in ws_summary[12]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = header_alignment

for cell in ws_summary[22]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = header_alignment

# ========== 工作表3：图表分析 ==========
ws_chart = wb.create_sheet("图表分析")

# 创建销售员销售额对比柱状图
chart1 = BarChart()
chart1.type = "col"
chart1.style = 10
chart1.title = "销售员销售额对比"
chart1.y_axis.title = "销售额"
chart1.x_axis.title = "销售员"

# 使用汇总统计表的数据
data_ref = Reference(ws_summary, min_col=3, min_row=3, max_row=8)
cats_ref = Reference(ws_summary, min_col=1, min_row=4, max_row=8)

chart1.add_data(data_ref, titles_from_data=True)
chart1.set_categories(cats_ref)

ws_chart.add_chart(chart1, "A1")

# 创建产品销售额饼图
chart2 = PieChart()
chart2.title = "产品销售额占比"

data_ref2 = Reference(ws_summary, min_col=3, min_row=12, max_row=17)
cats_ref2 = Reference(ws_summary, min_col=1, min_row=13, max_row=17)

chart2.add_data(data_ref2, titles_from_data=True)
chart2.set_categories(cats_ref2)

ws_chart.add_chart(chart2, "A20")

# ========== 保存文件 ==========
wb.save("完整销售报表系统.xlsx")
print("完整的销售报表系统已创建！")
print("包含：")
print("  1. 销售数据表（100条记录，已格式化）")
print("  2. 汇总统计表（按销售员、产品、地区汇总）")
print("  3. 图表分析表（柱状图和饼图）")
print("\n这个报表系统展示了openpyxl的综合应用能力！")


## 总结：OpenPyXL的优势和应用场景

### OpenPyXL vs 其他工具

| 特性 | OpenPyXL | Pandas | 直接操作Excel |
|------|----------|--------|---------------|
| **样式控制** | ⭐⭐⭐⭐⭐ 完全控制 | ⭐⭐ 有限支持 | ⭐⭐⭐⭐⭐ 完全控制 |
| **公式设置** | ⭐⭐⭐⭐⭐ 支持所有公式 | ⭐⭐ 有限支持 | ⭐⭐⭐⭐⭐ 完全支持 |
| **图表创建** | ⭐⭐⭐⭐ 支持多种图表 | ❌ 不支持 | ⭐⭐⭐⭐⭐ 完全支持 |
| **批量处理** | ⭐⭐⭐⭐⭐ 自动化批量 | ⭐⭐⭐⭐ 支持 | ⭐ 手动操作 |
| **数据验证** | ⭐⭐⭐⭐ 支持 | ❌ 不支持 | ⭐⭐⭐⭐⭐ 完全支持 |
| **学习曲线** | ⭐⭐⭐ 中等 | ⭐⭐⭐⭐ 简单 | ⭐⭐⭐⭐⭐ 无需学习 |
| **适用场景** | 报表生成、格式控制 | 数据分析 | 少量手动操作 |

### 最佳应用场景

1. **批量报表生成**：每月/每周需要生成大量格式化的报表
2. **模板填充**：基于标准模板快速生成多个文件
3. **数据清洗后导出**：处理后的数据需要专业的Excel格式
4. **自动化工作流**：将Excel操作集成到Python自动化流程中
5. **复杂格式要求**：需要精确控制样式、公式、图表等

### 学习建议

1. **从基础开始**：先掌握创建工作簿、写入数据、读取数据
2. **逐步深入**：然后学习样式设置、公式、批量操作
3. **实践应用**：通过实际项目巩固知识
4. **查阅文档**：遇到问题时查阅[官方文档](https://openpyxl.readthedocs.io/)

### 下一步学习

- 学习openpyxl的更多高级功能（数据透视表、宏等）
- 结合其他库（如pandas）进行数据分析后再用openpyxl导出
- 探索openpyxl的性能优化技巧（处理大文件）
- 学习Excel的其他Python库（xlwings、xlsxwriter等）进行对比

---

**恭喜你完成了OpenPyXL的学习！现在你可以用代码高效地处理Excel文件了！** 🎉
